**Requires**

**Output**

## Imports

In [1]:
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd

from compert.paths import DATA_DIR, PROJECT_DIR

sc.set_figure_params(dpi=80, frameon=False)
sc.logging.print_header()

scanpy==1.9.0.dev41+g58f4904c anndata==0.7.6 umap==0.5.1 numpy==1.19.2 scipy==1.6.2 pandas==1.2.4 scikit-learn==0.24.2 statsmodels==0.12.2 python-igraph==0.9.1 louvain==0.7.0 pynndescent==0.5.2


In [2]:
%load_ext autoreload
%autoreload 2

### Loading LINCS and reference data

In [3]:
full = True
load_adata = True 

if full:
    adata_in = DATA_DIR / 'lincs_full_pp.h5ad'
    adata_out =  PROJECT_DIR / 'datasets' / 'lincs_full_smiles.h5ad' 
else: 
    adata_in = DATA_DIR / 'lincs_pp.h5ad'
    adata_out = PROJECT_DIR / 'datasets' / 'lincs_smiles.h5ad'  

    
adata = sc.read(adata_in) if load_adata else None

Checking number of drugs for LINCS

In [4]:
pert_id_unique = pd.Series(np.unique(adata.obs.pert_id))
print(f"# of unique perturbations: {len(pert_id_unique)}")

# of unique perturbations: 18743


Loading reference dataframe that contains SMILES 
restricting to `'pert_id'` and `'canonical_smiles'`

In [5]:
reference_df = pd.read_csv(DATA_DIR /'GSE92742_Broad_LINCS_pert_info.txt', delimiter = "\t")
reference_df = reference_df.loc[reference_df.pert_id.isin(pert_id_unique), ['pert_id', 'canonical_smiles']]
reference_df.canonical_smiles.value_counts()

-666                                                                                                                                                             63
restricted                                                                                                                                                       14
COc1ccc(C[C@H](C)NC[C@@H](O)c2ccc(O)c(NC=O)c2)cc1                                                                                                                 2
CO[C@H]1\C=C\O[C@@]2(C)Oc3c(C2=O)c2c(O)c(\C=N\N4CCN(C)CC4)c(NC(=O)\C(C)=C/C=C/[C@H](C)[C@H](O)[C@@H](C)[C@@H](O)[C@@H](C)[C@H](OC(C)=O)[C@@H]1C)c(O)c2c(O)c3C     2
CCOC(=O)C1=C(NC(=C(C1C2=CC=CC=C2Cl)C(=O)OC)C)COCCN                                                                                                                2
                                                                                                                                                                 ..
COc1cccc(CNC(=O)

In [6]:
cond = ~pert_id_unique.isin(reference_df.pert_id)
print(f"From {len(pert_id_unique)} total drugs, {cond.sum()} were not part of the reference dataframe.")

From 18743 total drugs, 890 were not part of the reference dataframe.


Adding `'canoncical_smiles'` column to `adata.obs` via `pd.merge`

In [7]:
adata.obs = adata.obs.reset_index().merge(reference_df, how="left").set_index('index')

Removing invalid SMILES strings 

In [8]:
adata.obs.pert_id

index
REP.A001_A375_24H_X1_B22:A03-2             DMSO
REP.A001_A375_24H_X1_B22:A04-2             DMSO
REP.A001_A375_24H_X1_B22:A05-2             DMSO
REP.A001_A375_24H_X1_B22:A06-2             DMSO
REP.A001_A375_24H_X1_B22:A07-2    BRD-K25114078
                                      ...      
PCLB003_PC3_24H_X3_B13:P20-1      BRD-A75409952
PCLB003_PC3_24H_X3_B13:P21-1      BRD-A75409952
PCLB003_PC3_24H_X3_B13:P22-1      BRD-A75409952
PCLB003_PC3_24H_X3_B13:P23-1      BRD-A75409952
PCLB003_PC3_24H_X3_B13:P24-1      BRD-A75409952
Name: pert_id, Length: 1023036, dtype: object

In [9]:
reference_df

,pert_id,canonical_smiles
98,BRD-A00100033,CC1CS(=O)(=O)CCN1N=Cc1ccc(o1)[N+]([O-])=O
99,BRD-A00150179,NC(Cc1c[nH]c2cccc(O)c12)C(O)=O
100,BRD-A00267231,CCCCC#Cc1nc(NC)c2ncn(C3OC(CO)C(O)C3O)c2n1
101,BRD-A00420644,CCN1C2C(C(=NC2Nc3ccccc13)OC)c4ccccc4
102,BRD-A00474148,Oc1ccc(cc1)N1CCN(CC1)[S+]([O-])(=O)c1ccc2NC(=O...
...,...,...
26389,CMAP-PRISM-TP7,-666
26401,CMAP-T2DTUNICAMYCIN,-666
30250,DMSO,CS(=O)C
30365,H2O,-666


In [10]:
adata.obs.loc[:, 'canonical_smiles'] = adata.obs.canonical_smiles.astype('str')
invalid_smiles = adata.obs.canonical_smiles.isin(['-666', 'restricted', 'nan'])
print(f'Among {len(adata)} observations, {100*invalid_smiles.sum()/len(adata):.2f}% ({invalid_smiles.sum()}) have an invalid SMILES string')
adata = adata[~invalid_smiles]

Among 1023036 observations, 13.66% (139764) have an invalid SMILES string


Remove invalid `'pert_dose'` value: `-666`

In [11]:
cond = adata.obs.pert_dose.isin([-666])
adata = adata[~cond]
print(f"A total of {cond.sum()} observations have invalid dose values")

A total of 42592 observations have invalid dose values


In [12]:
drugs_validation = adata.obs.canonical_smiles.value_counts() < 6
valid_drugs = drugs_validation.index[~drugs_validation]
cond = adata.obs.canonical_smiles.isin(valid_drugs)
print(f"A total of {(~cond).sum()} observation belong to drugs which do not have enough replicates")
adata = adata[cond]

A total of 3 observation belong to drugs which do not have enough replicates


Checking that SMILES are valid according to `rdkit` 

In [13]:
from rdkit import Chem

def check_smiles(smiles):
    m = Chem.MolFromSmiles(smiles,sanitize=False)
    if m is None:
        print('invalid SMILES')
        return False
    else:
        try:
            Chem.SanitizeMol(m)
        except:
            print('invalid chemistry')
            return False
    return True

def remove_invalid_smiles(dataframe, smiles_key: str = 'SMILES', return_condition: bool = False):
    unique_drugs = pd.Series(np.unique(dataframe[smiles_key]))
    valid_drugs = unique_drugs.apply(check_smiles)
    print(f"A total of {(~valid_drugs).sum()} have invalid SMILES strings")
    _validation_map = dict(zip(unique_drugs, valid_drugs))
    cond = dataframe[smiles_key].apply(lambda x: _validation_map[x])
    if return_condition: 
        return cond
    dataframe = dataframe[cond].copy()
    return dataframe

adata

View of AnnData object with n_obs × n_vars = 840677 × 978
    obs: 'cell_id', 'det_plate', 'det_well', 'lincs_phase', 'pert_dose', 'pert_dose_unit', 'pert_id', 'pert_iname', 'pert_mfc_id', 'pert_time', 'pert_time_unit', 'pert_type', 'rna_plate', 'rna_well', 'condition', 'cell_type', 'dose_val', 'cov_drug_dose_name', 'control', 'split', 'canonical_smiles'
    var: 'pr_gene_title', 'pr_is_lm', 'pr_is_bing'
    uns: 'cydata_pull', 'rank_genes_groups_cov'

In [14]:
cond = remove_invalid_smiles(adata.obs, smiles_key='canonical_smiles', return_condition=True)
adata = adata[cond]

A total of 0 have invalid SMILES strings


### Add additional drugbank info to `adata.obs`

In [15]:
from os.path import exists

drugbank_path = DATA_DIR / 'drug_bank'/ 'drugbank_all.csv'
if exists(drugbank_path): 
    drugbank_df = pd.read_csv(drugbank_path)
else: 
    print(f'Invalid path: {drugbank_path}')

/home/icb/leon.hetzel/miniconda3/envs/py38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (54,62) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [16]:
from rdkit.Chem import CanonSmiles

drugs_canonical = pd.Series(np.unique(adata.obs.canonical_smiles)).apply(CanonSmiles)
db_canonical_smiles = drugbank_df.SMILES.apply(CanonSmiles)
n_overlap = drugs_canonical.isin(db_canonical_smiles).sum()
print(f'From a total of {len(drugs_canonical)}, {100*n_overlap/len(drugs_canonical):.2f}% ({n_overlap}) is also available in drugbank.')

From a total of 17767, 7.72% (1371) is also available in drugbank.


RDKit WARNING: [15:14:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [15:14:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [15:14:11] WARNING: not removing hydrogen atom without neighbors


In [17]:
cond = db_canonical_smiles.isin(drugs_canonical)
drugbank_df.loc[cond, ['ATC_level_1']].value_counts()

ATC_level_1                               
an                                            393
NERVOUS SYSTEM                                184
CARDIOVASCULAR SYSTEM                         152
ANTINEOPLASTIC AND IMMUNOMODULATING AGENTS     98
ALIMENTARY TRACT AND METABOLISM                93
ANTIINFECTIVES FOR SYSTEMIC USE                81
RESPIRATORY SYSTEM                             78
GENITO URINARY SYSTEM AND SEX HORMONES         60
MUSCULO-SKELETAL SYSTEM                        58
DERMATOLOGICALS                                51
SENSORY ORGANS                                 47
INSECTICIDES AND REPELLENTS                    29
VARIOUS                                        21
BLOOD AND BLOOD FORMING ORGANS                 17
EXCL. SEX HORMONES AND INSULINS                 7
dtype: int64

### Add `train`, `test`, `ood` split for full lincs dataset (if not already part in `adata.obs`)

In [18]:
from sklearn.model_selection import train_test_split

if 'split' not in list(adata.obs):
    print("Addig 'split' to 'adata.obs'.")
    unique_drugs = np.unique(adata.obs.canonical_smiles)
    drugs_train, drugs_tmp = train_test_split(unique_drugs, test_size=0.2)
    drugs_val, drugs_test = train_test_split(drugs_tmp, test_size=0.5)

    adata.obs['split'] = 'train'
    adata.obs.loc[adata.obs.canonical_smiles.isin(drugs_val), 'split'] = 'test'
    adata.obs.loc[adata.obs.canonical_smiles.isin(drugs_test), 'split'] = 'ood'

### Check that `.obs.split=='test'` has sufficient samples for `pert_id` and `cell_id`

In [19]:
adata.obs.split.value_counts()

train    699463
test     133383
ood        7831
Name: split, dtype: int64

In [20]:
cond_test = adata.obs.split.isin(['test'])
adata.obs.loc[cond_test, 'cell_id'].value_counts()

MCF7        18766
VCAP        17841
PC3         17414
A375        11188
HT29        10970
            ...  
HUES3          93
SNUC5          89
NCIH1694       86
SKMEL28        77
HS27A           9
Name: cell_id, Length: 82, dtype: int64

In [21]:
adata.obs.loc[cond_test, 'pert_id'].value_counts()

BRD-K60230970    904
BRD-K50691590    882
BRD-K81418486    694
DMSO             618
BRD-A19500257    538
                ... 
BRD-K79746073      1
BRD-K01751906      1
BRD-K10558099      1
BRD-K29360016      1
BRD-K23208702      1
Name: pert_id, Length: 15649, dtype: int64

In [22]:
pert_count_treshold = 5
cov_count_treshold = 20

pert_id_neg = adata.obs.loc[cond_test, 'pert_id'].value_counts() < pert_count_treshold
print(f"pert_id: {pert_id_neg.sum()}/{len(pert_id_neg)} converted back to 'train' due to insufficient # of samples.")

cov_id_neg = adata.obs.loc[cond_test, 'cell_id'].value_counts() < cov_count_treshold
print(f"cell_id: {cov_id_neg.sum()}/{len(cov_id_neg)} converted back to 'train' due to insufficient # of samples.")

cond = cond_test & adata.obs.pert_id.isin(pert_id_neg.index[pert_id_neg])
cond |= cond_test & adata.obs.cell_id.isin(cov_id_neg.index[cov_id_neg])

pert_id: 9257/15649 converted back to 'train' due to insufficient # of samples.
cell_id: 1/82 converted back to 'train' due to insufficient # of samples.


In [24]:
adata.obs['split1'] = adata.obs.split.copy()
adata.obs.loc[cond, 'split1'] = 'train'
print(f"split['test']: {cond.sum()}/{len(cond)} samples are converted back to 'train'.")

split['test']: 18885/840677 samples are converted back to 'train'.


In [25]:
adata.obs.split1.value_counts()

train    718348
test     114498
ood        7831
Name: split1, dtype: int64

### Add random split

In [26]:
adata.obs_names

Index(['REP.A001_A375_24H_X1_B22:B13-2', 'REP.A001_A375_24H_X1_B22:B14-2',
       'REP.A001_A375_24H_X1_B22:B15-2', 'REP.A001_A375_24H_X1_B22:B16-2',
       'REP.A001_A375_24H_X1_B22:B17-2', 'REP.A001_A375_24H_X1_B22:B18-2',
       'REP.A001_A375_24H_X1_B22:B19-2', 'REP.A001_A375_24H_X1_B22:B20-2',
       'REP.A001_A375_24H_X1_B22:B21-2', 'REP.A001_A375_24H_X1_B22:B22-2',
       ...
       'PCLB003_PC3_24H_X3_B13:P15-1', 'PCLB003_PC3_24H_X3_B13:P16-1',
       'PCLB003_PC3_24H_X3_B13:P17-1', 'PCLB003_PC3_24H_X3_B13:P18-1',
       'PCLB003_PC3_24H_X3_B13:P19-1', 'PCLB003_PC3_24H_X3_B13:P20-1',
       'PCLB003_PC3_24H_X3_B13:P21-1', 'PCLB003_PC3_24H_X3_B13:P22-1',
       'PCLB003_PC3_24H_X3_B13:P23-1', 'PCLB003_PC3_24H_X3_B13:P24-1'],
      dtype='object', name='index', length=840677)

In [27]:
train_obs, val_test_obs = train_test_split(adata.obs_names, test_size=0.15)
val_obs, test_obs = train_test_split(val_test_obs, test_size=0.5)

In [28]:
adata.obs['random_split'] = ''
adata.obs.loc[train_obs, 'random_split'] = 'train'
adata.obs.loc[val_obs, 'random_split'] = 'test'
adata.obs.loc[test_obs, 'random_split'] = 'ood'


adata.obs['random_split'].value_counts() 

train    714575
test      63051
ood       63051
Name: random_split, dtype: int64

Check that perturbations occur in train split (no explicit ood!)

In [29]:
len(adata.obs.loc[adata.obs.random_split == 'train', 'pert_id'].unique()) 

17775

In [30]:
len(adata.obs.pert_id.unique())

17775

## Safe adata

In [31]:
adata.write(adata_out)
adata

... storing 'pert_id' as categorical
... storing 'canonical_smiles' as categorical
... storing 'random_split' as categorical


AnnData object with n_obs × n_vars = 840677 × 978
    obs: 'cell_id', 'det_plate', 'det_well', 'lincs_phase', 'pert_dose', 'pert_dose_unit', 'pert_id', 'pert_iname', 'pert_mfc_id', 'pert_time', 'pert_time_unit', 'pert_type', 'rna_plate', 'rna_well', 'condition', 'cell_type', 'dose_val', 'cov_drug_dose_name', 'control', 'split', 'canonical_smiles', 'split1', 'random_split'
    var: 'pr_gene_title', 'pr_is_lm', 'pr_is_bing'
    uns: 'cydata_pull', 'rank_genes_groups_cov'

In [32]:
# adata.obs['drug_dose_name']

### Loading the result for `adata_out`

In [33]:
adata = sc.read(adata_out)

**Additional**: Check that `adata.uns[rank_genes_groups_cov]` has all entries in `adata.obs.cov_drug_dose_name` as keys

In [34]:
for i, k in enumerate(adata.obs.cov_drug_dose_name.unique()):
    try: 
        adata.uns['rank_genes_groups_cov'][k]
    except: 
        print(f"{i}: {k}") if 'DMSO' not in k else None